In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lvmdrp.scripts import preproc


NAME = "dummy-desi-fiberflat.fits.gz"

import re


type2fmt = {
    "bool": "L",
    "binary": "L",
    "logical": "L",
    "byte": "X",
    "char": "A",
    "int16": "I",
    "int32": "J",
    "int64": "K",
    "float32": "E",
    "float64": "D"
}
type2dmy = dict(zip(type2fmt.keys(),[True, True, True, 0, "x", -9999, -9999, -9999, -9999.9, -9999.9]))

pattern = re.compile(r"\[(\d+)\]")

In [2]:
import pandas as pd

data_url = "https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/exposures/NIGHT/EXPID/fiberflat-CAMERA-EXPID.html?highlight=fiberflat"

desimodel_tables = []
for table in pd.read_html(data_url):
    clean_table = table.fillna(-9999)
    clean_table = clean_table.applymap(lambda s: s.replace("’","").replace("‘","").replace("…","") if type(s)==str else s)
    clean_table = clean_table.applymap(lambda s: pd.to_numeric(s, errors="ignore", downcast="integer"))
    desimodel_tables.append(clean_table)

In [3]:
desimodel_tables[1]

,Number,EXTNAME,Type,Contents
0,HDU0,FIBERFLAT,IMAGE,"fiberflat[nspec, nwave]"
1,HDU1,IVAR,IMAGE,inverse variance of fiberflat
2,HDU2,MASK,IMAGE,bitmask of fiberflat (0=good)
3,HDU3,MEANSPEC,IMAGE,average spectrum[nwave]
4,HDU4,WAVELENGTH,IMAGE,wavelength grid[nwave] in Angstroms
5,HDU5,FIBERMAP,BINTABLE,fibermap


In [4]:
desimodel_tables[7]

,KEY,Example Value,Type,Comment
0,NAXIS1,357,int,length of dimension 1
1,NAXIS2,500,int,length of dimension 2
2,ENCODING,ascii,str,-9999
3,CHECKSUM,GXdcHVbZGVbbGVbZ,str,HDU checksum updated 2020-04-29T21:19:14
4,DATASUM,1416756853,str,data unit checksum updated 2020-04-29T21:19:14


In [5]:
# fiberflats
# arcs

In [6]:
dummy = fits.HDUList()

for k, j in enumerate(range(2,7)):

    hdu = fits.PrimaryHDU(header=fits.Header(cards=map(tuple, desimodel_tables[j].drop(columns="Type").values.tolist())), data=np.zeros((2048,2048)))
    hdu.name = desimodel_tables[1].loc[k,"EXTNAME"]
    dummy.append(hdu)


k, j = 5, 7
header = fits.Header(cards=map(tuple, desimodel_tables[j].drop(columns="Type").values.tolist()))
columns = []
for i, row in desimodel_tables[j+1].iterrows():

    shape = re.findall(pattern, row.Type)
    ctype = re.sub(pattern, "", row.Type)
    if len(shape) != 0:
        fmt = "%s%s"%(shape[0],type2fmt[ctype])
        arr = int(shape[0])*[type2dmy[ctype]] if ctype!="char" else [int(shape[0])*type2dmy[ctype]]
    else:
        fmt = type2fmt[ctype]
        arr = [type2dmy[ctype]]

    columns.append(fits.Column(name=row.Name, format=fmt, array=arr, unit=row.Units if row.Units!=-9999 else None))

hdu = fits.BinTableHDU.from_columns(header=header, columns=columns)
hdu.name = desimodel_tables[1].loc[k,"EXTNAME"]
dummy.append(hdu)

dummy.writeto(os.path.join(NAME), overwrite=True)
dummy.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  FIBERFLAT     1 PrimaryHDU      20   (2048, 2048)   float64   
  1  IVAR          1 ImageHDU        10   (2048, 2048)   float64   
  2  MASK          1 ImageHDU        10   (2048, 2048)   float64   
  3  MEANSPEC      1 ImageHDU        10   (2048, 2048)   float64   
  4  WAVELENGTH    1 ImageHDU        11   (2048, 2048)   float64   
  5  FIBERMAP      1 BinTableHDU    160   1R x 73C   ['K', 'K', 'K', 'K', 'K', 'D', 'D', 'D', 'D', 'K', 'K', '4A', 'J', 'D', 'K', 'E', 'E', 'E', 'E', 'E', 'I', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '1A', 'J', 'K', 'K', 'J', 'K', 'J', 'J', 'J', 'J', 'J', '3A', 'E', 'E', 'E', 'K', 'L', 'I', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'J', 'J']   


In [19]:
dummy.writeto(os.path.join(os.environ["DESI_SPECTRO_CALIB"]+"/spec/sp0","fiberflat-z0.fits"), overwrite=True)